# Strategies for improving neural network performance 


[relative link](https://cambridgespark.com/content/tutorials/neural-networks-tuning-techniques/index.html)

1. L2 regularisation
2. Initialisation
3. Batch normalisation
4. Data augmentation
5. Ensemble methods
6. Early stopping

In [1]:
from keras.datasets import mnist # subroutines for fetching the MNIST dataset
from keras.models import Model # basic class for specifying and training a neural network
from keras.layers import Input, Dense, Flatten, Convolution2D, MaxPooling2D, Dropout
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values

batch_size = 128 # in each iteration, we consider 128 training examples at once
num_epochs = 12 # we iterate twelve times over the entire training set
kernel_size = 3 # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth = 32 # use 32 kernels in both convolutional layers
drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
hidden_size = 128 # there will be 128 neurons in both hidden layers

num_train = 60000 # there are 60000 training examples in MNIST
num_test = 10000 # there are 10000 test examples in MNIST

height, width, depth = 28, 28, 1 # MNIST images are 28x28 and greyscale
num_classes = 10 # there are 10 classes (1 per digit)

(X_train, y_train), (X_test, y_test) = mnist.load_data() # fetch MNIST data

X_train = X_train.reshape(X_train.shape[0], height, width, depth)
X_test = X_test.reshape(X_test.shape[0], height, width, depth)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255 # Normalise data to [0, 1] range
X_test /= 255 # Normalise data to [0, 1] range

Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
Y_test = np_utils.to_categorical(y_test, num_classes) # One-hot encode the labels

inp = Input(shape=(height, width, depth)) # N.B. TensorFlow back-end expects channel dimension last
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth, (kernel_size, kernel_size), padding='same', activation='relu')(inp)
conv_2 = Convolution2D(conv_depth, (kernel_size, kernel_size), padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
flat = Flatten()(drop_1)
hidden = Dense(hidden_size, activation='relu')(flat) # Hidden ReLU layer
drop = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop) # Output softmax layer

model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers

model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

model.fit(X_train, Y_train, # Train the model using the training set...
          batch_size=batch_size, epochs=num_epochs,
          verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation
model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

/opt/conda/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Train on 54000 samples, validate on 6000 samples
Epoch 1/12
54000/54000 [==============================] - 56s 1ms/step - loss: 0.2944 - acc: 0.9097 - val_loss: 0.0701 - val_acc: 0.9797
Epoch 2/12
54000/54000 [==============================] - 55s 1ms/step - loss: 0.1068 - acc: 0.9683 - val_loss: 0.0499 - val_acc: 0.9855
Epoch 3/12
54000/54000 [==============================] - 55s 1ms/step - loss: 0.0788 - acc: 0.9753 - val_loss: 0.0401 - val_acc: 0.9892
Epoch 4/12
54000/54000 [==============================] - 55s 1ms/step - loss: 0.0625 - acc: 0.9806 - val_loss: 0.0375 - val_acc: 0.9888
Epoch 5/12
54000/54000 [==============================] - 56s 1ms/step - loss: 0.0579 - acc: 0.9826 - val_loss: 0.0328 - val_acc: 0.9903
Epoch 6/12
54000/54000 [==============================] - 55s 1ms/step - loss: 0.0474 - acc: 0.9856 - val_loss: 0.0359 - val_acc: 0.9900
Epoch 7/12
54000/54000 [==============================] - 55s 1ms/step - loss: 0.0414 - acc: 0.9871 - val_loss: 0.0333 - val_acc:

[0.028896287516362598, 0.9916]

L2 regularisation

In [2]:
from keras.regularizers import l2 # L2-regularisation
l2_lambda = 0.0001
conv_1 = Convolution2D(conv_depth, (kernel_size, kernel_size), 
    padding='same', 
    kernel_regularizer=l2(l2_lambda), 
    activation='relu')(inp)
conv_2 = Convolution2D(conv_depth, (kernel_size, kernel_size), padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
flat = Flatten()(drop_1)
hidden = Dense(hidden_size, activation='relu')(flat) # Hidden ReLU layer
drop = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop) # Output softmax layer

model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers

model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

model.fit(X_train, Y_train, # Train the model using the training set...
          batch_size=batch_size, epochs=num_epochs,
          verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation
model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

Train on 54000 samples, validate on 6000 samples
Epoch 1/12
54000/54000 [==============================] - 55s 1ms/step - loss: 0.2841 - acc: 0.9136 - val_loss: 0.0618 - val_acc: 0.9832
Epoch 2/12
54000/54000 [==============================] - 55s 1ms/step - loss: 0.1032 - acc: 0.9694 - val_loss: 0.0452 - val_acc: 0.9878
Epoch 3/12
54000/54000 [==============================] - 55s 1ms/step - loss: 0.0733 - acc: 0.9778 - val_loss: 0.0379 - val_acc: 0.9892
Epoch 4/12
54000/54000 [==============================] - 55s 1ms/step - loss: 0.0632 - acc: 0.9807 - val_loss: 0.0380 - val_acc: 0.9900
Epoch 5/12
54000/54000 [==============================] - 55s 1ms/step - loss: 0.0536 - acc: 0.9831 - val_loss: 0.0364 - val_acc: 0.9918
Epoch 6/12
54000/54000 [==============================] - 55s 1ms/step - loss: 0.0464 - acc: 0.9856 - val_loss: 0.0349 - val_acc: 0.9903
Epoch 7/12
54000/54000 [==============================] - 55s 1ms/step - loss: 0.0419 - acc: 0.9867 - val_loss: 0.0345 - val_acc:

[0.027927689286880195, 0.9924]

Add initialisation to a layer


In [3]:
# Add initialisation to a layer
conv_1 = Convolution2D(conv_depth, (kernel_size, kernel_size), 
    padding='same', 
    kernel_initializer='he_uniform', 
    kernel_regularizer=l2(l2_lambda), 
    activation='relu')(inp)
conv_2 = Convolution2D(conv_depth, (kernel_size, kernel_size), padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
flat = Flatten()(drop_1)
hidden = Dense(hidden_size, activation='relu')(flat) # Hidden ReLU layer
drop = Dropout(drop_prob_2)(hidden)
# Add Xavier initialisation to a layer
out = Dense(num_classes, 
    kernel_initializer='glorot_uniform', 
    kernel_regularizer=l2(l2_lambda), 
    activation='softmax')(drop)

model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers

model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

model.fit(X_train, Y_train, # Train the model using the training set...
          batch_size=batch_size, epochs=num_epochs,
          verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation
model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

Train on 54000 samples, validate on 6000 samples
Epoch 1/12
54000/54000 [==============================] - 55s 1ms/step - loss: 0.2579 - acc: 0.9224 - val_loss: 0.0537 - val_acc: 0.9872
Epoch 2/12
54000/54000 [==============================] - 54s 1ms/step - loss: 0.0992 - acc: 0.9719 - val_loss: 0.0513 - val_acc: 0.9883
Epoch 3/12
54000/54000 [==============================] - 55s 1ms/step - loss: 0.0778 - acc: 0.9786 - val_loss: 0.0445 - val_acc: 0.9883
Epoch 4/12
54000/54000 [==============================] - 55s 1ms/step - loss: 0.0647 - acc: 0.9823 - val_loss: 0.0455 - val_acc: 0.9903
Epoch 5/12
54000/54000 [==============================] - 55s 1ms/step - loss: 0.0571 - acc: 0.9845 - val_loss: 0.0426 - val_acc: 0.9915
Epoch 6/12
54000/54000 [==============================] - 55s 1ms/step - loss: 0.0512 - acc: 0.9865 - val_loss: 0.0395 - val_acc: 0.9908
Epoch 7/12
54000/54000 [==============================] - 55s 1ms/step - loss: 0.0462 - acc: 0.9876 - val_loss: 0.0384 - val_acc:

[0.034956194722652434, 0.9912]

In [4]:
from keras.layers.normalization import BatchNormalization # batch normalisation
# ...
inp_norm = BatchNormalization()(inp) # apply BN to the input (N.B. need to rename here)
conv_1 = Convolution2D(conv_depth, (kernel_size, kernel_size), 
    padding='same', 
    kernel_initializer='he_uniform', 
    kernel_regularizer=l2(l2_lambda), 
    activation='relu')(inp_norm)
conv_1 = BatchNormalization()(conv_1) # apply BN to the first conv layer
conv_2 = Convolution2D(conv_depth, (kernel_size, kernel_size), padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
flat = Flatten()(drop_1)
hidden = Dense(hidden_size, activation='relu')(flat) # Hidden ReLU layer
drop = Dropout(drop_prob_2)(hidden)
# Add Xavier initialisation to a layer
out = Dense(num_classes, 
    kernel_initializer='glorot_uniform', 
    kernel_regularizer=l2(l2_lambda), 
    activation='softmax')(drop)

model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers

model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

model.fit(X_train, Y_train, # Train the model using the training set...
          batch_size=batch_size, epochs=num_epochs,
          verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation
model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

Train on 54000 samples, validate on 6000 samples
Epoch 1/12
54000/54000 [==============================] - 134s 2ms/step - loss: 0.4087 - acc: 0.8777 - val_loss: 0.0657 - val_acc: 0.9825
Epoch 2/12
54000/54000 [==============================] - 201s 4ms/step - loss: 0.1709 - acc: 0.9497 - val_loss: 0.0590 - val_acc: 0.9848
Epoch 3/12
54000/54000 [==============================] - 208s 4ms/step - loss: 0.1356 - acc: 0.9603 - val_loss: 0.0479 - val_acc: 0.9883
Epoch 4/12
54000/54000 [==============================] - 214s 4ms/step - loss: 0.1134 - acc: 0.9676 - val_loss: 0.0463 - val_acc: 0.9890
Epoch 5/12
54000/54000 [==============================] - 206s 4ms/step - loss: 0.0935 - acc: 0.9732 - val_loss: 0.0448 - val_acc: 0.9892
Epoch 6/12
54000/54000 [==============================] - 226s 4ms/step - loss: 0.0828 - acc: 0.9763 - val_loss: 0.0464 - val_acc: 0.9903
Epoch 7/12
54000/54000 [==============================] - 220s 4ms/step - loss: 0.0719 - acc: 0.9795 - val_loss: 0.0422 - v

[0.0403479166328907, 0.9908]

Data Augumentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator # data augmentation
# ... after model.compile(...)
# Explicitly split the training and validation sets
X_val = X_train[54000:]
Y_val = Y_train[54000:]
X_train = X_train[:54000]
Y_train = Y_train[:54000]

inp = Input(shape=(height, width, depth)) # N.B. TensorFlow back-end expects channel dimension last
inp_norm = BatchNormalization()(inp) # Apply BN to the input (N.B. need to rename here)
conv_1 = Convolution2D(conv_depth, (kernel_size, kernel_size), 
    padding='same', 
    kernel_initializer='he_uniform', 
    kernel_regularizer=l2(l2_lambda), 
    activation='relu')(inp_norm)
conv_1 = BatchNormalization()(conv_1) # apply BN to the first conv layer
conv_2 = Convolution2D(conv_depth, (kernel_size, kernel_size), padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
flat = Flatten()(drop_1)
hidden = Dense(hidden_size, activation='relu')(flat) # Hidden ReLU layer
drop = Dropout(drop_prob_2)(hidden)
# Add Xavier initialisation to a layer
out = Dense(num_classes, 
    kernel_initializer='glorot_uniform', 
    kernel_regularizer=l2(l2_lambda), 
    activation='softmax')(drop)

model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers

model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

datagen = ImageDataGenerator(
            width_shift_range=0.1, # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1) # randomly shift images vertically (fraction of total height)
datagen.fit(X_train)

# fit the model on the batches generated by datagen.flow() - most parameters similar to model.fit
model.fit_generator(datagen.flow(X_train, Y_train,
                        batch_size=batch_size),
                        steps_per_epoch=X_train.shape[0],
                        epochs=num_epochs,
                        validation_data=(X_val, Y_val),
                        verbose=1)
model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

# All-in-one

In [ ]:
from keras.datasets import mnist # subroutines for fetching the MNIST dataset
from keras.models import Model # basic class for specifying and training a neural network
from keras.layers import Input, Dense, Flatten, Convolution2D, MaxPooling2D, Dropout, average
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values
from keras.regularizers import l2 # L2-regularisation
from keras.layers.normalization import BatchNormalization # batch normalisation
from keras.preprocessing.image import ImageDataGenerator # data augmentation
from keras.callbacks import EarlyStopping # early stopping

batch_size = 128 # in each iteration, we consider 128 training examples at once
num_epochs = 50 # we iterate at most fifty times over the entire training set
kernel_size = 3 # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth = 32 # use 32 kernels in both convolutional layers
drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
hidden_size = 128 # there will be 128 neurons in both hidden layers
l2_lambda = 0.0001 # use 0.0001 as a L2-regularisation factor
ens_models = 3 # we will train three separate models on the data

num_train = 60000 # there are 60000 training examples in MNIST
num_test = 10000 # there are 10000 test examples in MNIST

height, width, depth = 28, 28, 1 # MNIST images are 28x28 and greyscale
num_classes = 10 # there are 10 classes (1 per digit)

(X_train, y_train), (X_test, y_test) = mnist.load_data() # fetch MNIST data

X_train = X_train.reshape(X_train.shape[0], height, width, depth)
X_test = X_test.reshape(X_test.shape[0], height, width, depth)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
Y_test = np_utils.to_categorical(y_test, num_classes) # One-hot encode the labels

# Explicitly split the training and validation sets
X_val = X_train[54000:]
Y_val = Y_train[54000:]
X_train = X_train[:54000]
Y_train = Y_train[:54000]

inp = Input(shape=(height, width, depth)) # N.B. TensorFlow back-end expects channel dimension last
inp_norm = BatchNormalization()(inp) # Apply BN to the input (N.B. need to rename here)

outs = [] # the list of ensemble outputs
for i in range(ens_models):
    # Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer), applying BN in between
    conv_1 = Convolution2D(conv_depth, (kernel_size, kernel_size), 
        padding='same', 
        kernel_initializer='he_uniform', 
        kernel_regularizer=l2(l2_lambda), 
        activation='relu')(inp_norm)
    conv_1 = BatchNormalization()(conv_1)
    conv_2 = Convolution2D(conv_depth, (kernel_size, kernel_size), 
        padding='same', 
        kernel_initializer='he_uniform', 
        kernel_regularizer=l2(l2_lambda), 
        activation='relu')(conv_1)
    conv_2 = BatchNormalization()(conv_2)
    pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    flat = Flatten()(drop_1)
    hidden = Dense(hidden_size, 
        kernel_initializer='he_uniform', 
        kernel_regularizer=l2(l2_lambda), 
        activation='relu')(flat) # Hidden ReLU layer
    hidden = BatchNormalization()(hidden)
    drop = Dropout(drop_prob_2)(hidden)
    outs.append(Dense(num_classes, 
        kernel_initializer='glorot_uniform', 
        kernel_regularizer=l2(l2_lambda), 
        activation='softmax')(drop)) # Output softmax layer

out = average(outs) # average the predictions to obtain the final output

model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers

model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

datagen = ImageDataGenerator(
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1)  # randomly shift images vertically (fraction of total height)
datagen.fit(X_train)

# fit the model on the batches generated by datagen.flow() - most parameters similar to model.fit
model.fit_generator(datagen.flow(X_train, Y_train,
                        batch_size=batch_size),
                        steps_per_epoch=X_train.shape[0],
                        epochs=num_epochs,
                        validation_data=(X_val, Y_val),
                        verbose=1,
                        callbacks=[EarlyStopping(monitor='val_loss', patience=5)]) # adding early stopping

model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

Epoch 1/50
54000/54000 [==============================] - 57813s 1s/step - loss: 0.1664 - acc: 0.9850 - val_loss: 0.1315 - val_acc: 0.9923
Epoch 2/50
54000/54000 [==============================] - 57384s 1s/step - loss: 0.1519 - acc: 0.9867 - val_loss: 0.1242 - val_acc: 0.9933
Epoch 3/50
54000/54000 [==============================] - 57596s 1s/step - loss: 0.1522 - acc: 0.9868 - val_loss: 0.1290 - val_acc: 0.9927
Epoch 4/50
54000/54000 [==============================] - 57734s 1s/step - loss: 0.1523 - acc: 0.9868 - val_loss: 0.1284 - val_acc: 0.9912
Epoch 5/50
54000/54000 [==============================] - 58196s 1s/step - loss: 0.1527 - acc: 0.9867 - val_loss: 0.1307 - val_acc: 0.9920
Epoch 6/50
54000/54000 [==============================] - 57692s 1s/step - loss: 0.1530 - acc: 0.9867 - val_loss: 0.1341 - val_acc: 0.9922
Epoch 7/50
44777/54000 [=======================>......] - ETA: 2:43:41 - loss: 0.1529 - acc: 0.9868